In [1]:
import pandas as pd
from pydantic import SecretStr

from langchain.docstore.document import Document
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA

from langchain.tools import tool, Tool
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType

from environments import OPENAI_API_KEY


In [2]:
if not OPENAI_API_KEY:
    raise ValueError("OpenAI Key not found")

openai_api_secret = SecretStr(OPENAI_API_KEY)

In [3]:
df = pd.read_csv("docs/datos_agricolas_3000_registros.csv")
df["Fecha"] = pd.to_datetime(df["Fecha"])

documentos = []
for _, row in df.iterrows():
    texto = f"""
    Fecha: {row['Fecha']}, Temporada: {row['Temporada']}, Cultivo: {row['Cultivo']}, Parcela: {row['Parcela']},
    Fase Productiva: {row['Fase Productiva']}, Cantidad (kg): {row['Cantidad (kg)']}, Precio por kg: {row['Precio por kg']},
    Ingresos: {row['Ingresos generados']}, Costos: {row['Costo Total']}, Rentabilidad: {row['Rentabilidad']},
    Clima: {row['Clima']}, Demanda de Mercado: {row['Demanda de Mercado']},
    Proyección Precio Temporada Siguiente: {row['Proyección Precio Temporada Siguiente']},
    Observaciones: {row['Observaciones']}, Línea Productiva: {row['Línea Productiva']}
    """.strip()
    documentos.append(texto)

# Preparación de los documentos para el RAG
docs = [Document(page_content=texto) for texto in documentos]

# Vectorización de los documentos con OpenAI
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_secret.get_secret_value())

In [4]:
vectorstore = Chroma.from_documents(docs, embeddings, persist_directory="./chroma_rentabilidad")

In [5]:
# Cargar vectorstore como retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# Crear chain de pregunta-respuesta
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-4"),
    retriever=retriever,
    return_source_documents=True
)

In [6]:
pregunta = "¿Cual fue la rentabilidad más alta de la Parcela A en 2023?"
respuesta = qa_chain(pregunta)
respuesta

/tmp/ipykernel_96293/1227950309.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  respuesta = qa_chain(pregunta)


{'query': '¿Cual fue la rentabilidad más alta de la Parcela A en 2023?',
 'result': 'La rentabilidad más alta de la Parcela A en 2023 fue de 399.54.',
 'source_documents': [Document(metadata={}, page_content='Fecha: 2023-04-29 00:00:00, Temporada: 2023-Verano, Cultivo: Trigo, Parcela: Parcela A,\n    Fase Productiva: Cosecha, Cantidad (kg): 982, Precio por kg: 0.83,\n    Ingresos: 815.06, Costos: 494, Rentabilidad: 321.06,\n    Clima: Lluvioso, Demanda de Mercado: Alta,\n    Proyección Precio Temporada Siguiente: 0.95,\n    Observaciones: Actividad de cosecha bajo clima lluvioso, relacionado con parcela Parcela A., Línea Productiva: Grano'),
  Document(metadata={}, page_content='Fecha: 2023-05-07 00:00:00, Temporada: 2023-Verano, Cultivo: Trigo, Parcela: Parcela A,\n    Fase Productiva: Mantenimiento, Cantidad (kg): 0, Precio por kg: 0.0,\n    Ingresos: 0.0, Costos: 334, Rentabilidad: -334.0,\n    Clima: Lluvioso, Demanda de Mercado: Alta,\n    Proyección Precio Temporada Siguiente: 0.

In [7]:
pregunta = "¿Cual fue la rentabilidad más alta de la Parcela A en 2023?"
respuesta = qa_chain(pregunta)
respuesta

{'query': '¿Cual fue la rentabilidad más alta de la Parcela A en 2023?',
 'result': 'La rentabilidad más alta de la Parcela A en 2023 fue de 399.54.',
 'source_documents': [Document(metadata={}, page_content='Fecha: 2023-04-29 00:00:00, Temporada: 2023-Verano, Cultivo: Trigo, Parcela: Parcela A,\n    Fase Productiva: Cosecha, Cantidad (kg): 982, Precio por kg: 0.83,\n    Ingresos: 815.06, Costos: 494, Rentabilidad: 321.06,\n    Clima: Lluvioso, Demanda de Mercado: Alta,\n    Proyección Precio Temporada Siguiente: 0.95,\n    Observaciones: Actividad de cosecha bajo clima lluvioso, relacionado con parcela Parcela A., Línea Productiva: Grano'),
  Document(metadata={}, page_content='Fecha: 2023-05-07 00:00:00, Temporada: 2023-Verano, Cultivo: Trigo, Parcela: Parcela A,\n    Fase Productiva: Mantenimiento, Cantidad (kg): 0, Precio por kg: 0.0,\n    Ingresos: 0.0, Costos: 334, Rentabilidad: -334.0,\n    Clima: Lluvioso, Demanda de Mercado: Alta,\n    Proyección Precio Temporada Siguiente: 0.

# Incorporación de Tools

In [ ]:
# import pandas as pd
# from dotenv import load_dotenv
# from pydantic import SecretStr

# from langchain.docstore.document import Document
# from langchain.vectorstores import Chroma
# from langchain_openai import OpenAIEmbeddings, ChatOpenAI
# from langchain.chains import RetrievalQA

# from langchain.tools import tool, Tool
# from langchain.agents import initialize_agent
# from langchain.agents.agent_types import AgentType

# from environments import OPENAI_API_KEY

# # Validar API Key
# if not OPENAI_API_KEY:
#     raise ValueError("OpenAI Key not found")
# openai_api_secret = SecretStr(OPENAI_API_KEY)

# # Cargar el CSV
# df = pd.read_csv("docs/datos_agricolas_3000_registros.csv")
# df["Fecha"] = pd.to_datetime(df["Fecha"])

# # Crear documentos para el RAG
# documentos = []
# for _, row in df.iterrows():
#     texto = f"""
#     Fecha: {row['Fecha']}, Temporada: {row['Temporada']}, Cultivo: {row['Cultivo']}, Parcela: {row['Parcela']},
#     Fase Productiva: {row['Fase Productiva']}, Cantidad (kg): {row['Cantidad (kg)']}, Precio por kg: {row['Precio por kg']},
#     Ingresos: {row['Ingresos generados']}, Costos: {row['Costo Total']}, Rentabilidad: {row['Rentabilidad']},
#     Clima: {row['Clima']}, Demanda de Mercado: {row['Demanda de Mercado']},
#     Proyección Precio Temporada Siguiente: {row['Proyección Precio Temporada Siguiente']},
#     Observaciones: {row['Observaciones']}, Línea Productiva: {row['Línea Productiva']}
#     """.strip()
#     documentos.append(Document(page_content=texto))

# # Embeddings y vectorstore
# embeddings = OpenAIEmbeddings(openai_api_key=openai_api_secret.get_secret_value())
# vectorstore = Chroma.from_documents(documentos, embeddings, persist_directory="./chroma_rentabilidad")
# retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# Crear RAG chain
llm = ChatOpenAI(model_name="gpt-4", temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# 🔧 Tool: Rentabilidad por parcela
@tool
def rentabilidad_por_parcela(parcela: str, anio: int) -> str:
    """Calcula la rentabilidad total de una parcela en un año específico."""
    df = pd.read_csv("docs/datos_agricolas_3000_registros.csv")
    df["Fecha"] = pd.to_datetime(df["Fecha"])
    
    # Normalizar nombre de parcela si es "A" → "Parcela A"
    if not parcela.lower().startswith("parcela"):
        parcela = f"Parcela {parcela.upper()}"

    df_filtrado = df[(df["Parcela"] == parcela) & (df["Fecha"].dt.year == anio)]
    total = df_filtrado["Rentabilidad"].sum()
    return f"La rentabilidad total de la parcela {parcela} en {anio} fue de {total:.2f}"

# Tool: RAG como herramienta semántica
retriever_tool = Tool(
    name="consultar_datos_agricolas",
    func=qa_chain.invoke,
    description="Usa esto para buscar información general del cultivo, clima, observaciones o condiciones de mercado."
)

# Inicializar agente con ambas tools
tools = [rentabilidad_por_parcela, retriever_tool]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)



/tmp/ipykernel_96293/139239859.py:75: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [9]:
# Probar una pregunta
pregunta = "¿Cuál fue la rentabilidad total de la Parcela A en 2023?"
respuesta = agent.invoke(pregunta)

# Mostrar la respuesta
print("Pregunta:", pregunta)
print("Respuesta:", respuesta)




> Entering new AgentExecutor chain...

Invoking: `rentabilidad_por_parcela` with `{'parcela': 'Parcela A', 'anio': 2023}`


La rentabilidad total de la parcela Parcela A en 2023 fue de 6776.89La rentabilidad total de la Parcela A en 2023 fue de 6776.89.

> Finished chain.
Pregunta: ¿Cuál fue la rentabilidad total de la Parcela A en 2023?
Respuesta: {'input': '¿Cuál fue la rentabilidad total de la Parcela A en 2023?', 'output': 'La rentabilidad total de la Parcela A en 2023 fue de 6776.89.'}


In [10]:
# Probar una pregunta
pregunta = "¿Cuál fue la parcela más rentable en 2023?"
respuesta = agent.invoke(pregunta)

# Mostrar la respuesta
print("Pregunta:", pregunta)
print("Respuesta:", respuesta)




> Entering new AgentExecutor chain...

Invoking: `rentabilidad_por_parcela` with `{'parcela': 'A', 'anio': 2023}`


La rentabilidad total de la parcela Parcela A en 2023 fue de 6776.89
Invoking: `rentabilidad_por_parcela` with `{'parcela': 'B', 'anio': 2023}`


La rentabilidad total de la parcela Parcela B en 2023 fue de -14034.90
Invoking: `rentabilidad_por_parcela` with `{'parcela': 'C', 'anio': 2023}`


La rentabilidad total de la parcela Parcela C en 2023 fue de 34361.80La parcela más rentable en 2023 fue la Parcela C con una rentabilidad total de 34361.80.

> Finished chain.
Pregunta: ¿Cuál fue la parcela más rentable en 2023?
Respuesta: {'input': '¿Cuál fue la parcela más rentable en 2023?', 'output': 'La parcela más rentable en 2023 fue la Parcela C con una rentabilidad total de 34361.80.'}
